1. Setup Mario

In [8]:
!pip install gym_super_mario_bros nes_py gymnasium "stable-baselines3[extra]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 2.0 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 MB 377.4 kB/s eta 0:00:00m eta 0:00:010:00:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 878.5 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 936.6 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 MB 579.1 kB/s eta 0:00:00m eta 0:00:010:00:04
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 kB 892.8 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━

In [9]:
!pip install stable-baselines3

  Using cached stable_baselines3-1.8.0-py3-none-any.whl (174 kB)
  Using cached gym-0.21.0.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached torch-2.0.0-cp311-none-macosx_10_9_x86_64.whl (139.5 MB)
  Using cached pandas-2.0.0-cp311-cp311-macosx_10_9_x86_64.whl (11.6 MB)
  Using cached matplotlib-3.7.1-cp311-cp311-macosx_10_12_x86_64.whl (7.4 MB)
  Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
  Using cached filelock-3.11.0-py3-none-any.whl (10.0 kB)
  Using cached sympy-1.11.1-py3-none-any.whl (6.5 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached contourpy-1.0.7-cp311-cp311-macosx_10_9_x86_64.whl (244 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.39.3-py3-none-any.whl (1.0 MB)
  Using cached kiwisolver-1.4.4-cp311-cp311-macosx_10_9_x86_64.whl (65 kB)
  Using cached Pillow-9.5.0-cp311-cp311-macosx_10_10_x86_64.whl (3.4 MB)
  Using 

In [15]:
!pip install opencv-python

  Using cached opencv-python-4.7.0.72.tar.gz (91.1 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for opencv-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [791 lines of output]
      
      
      --------------------------------------------------------------------------------
      -- Trying 'Ninja' generator
      --------------------------------
      ---------------------------
      ----------------------
      -----------------
      ------------
      -------
      --
      CMake Error: CMake was unable to find a build program corresponding to "Ninja".  CMAKE_MAKE_PROGRAM is not set.  You probably need to select a different build tool.Not searching for unused variables given on the command line.
      
      -- Configuring incomplete, errors occurred!
      --
      -------
      ------------
      ----

In [2]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym
# import gymnasium as gym



# #import the game
# import gym_super_mario_bros
# #import the joypad wrapper
# from nes_py.wrappers import JoypadSpace
# #import the SIMPLIFIED controls
# from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [3]:
env = gym.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="human")
# env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [4]:
#Create a flag - restart or not
done = True
#loop through each frame in the game
for step in range(5):
    #start the game to begin with
    if done:
        env.reset()
    #do random actions
    # action = env.action_space.sample()
    # print(action)env.action_space.sample()
    # print(len(env.step(env.action_space.sample()))
    # state, reward, done, info = env.step(env.action_space.sample())
    state, reward, done, truncated, info = env.step(env.action_space.sample())
    #show the game on the screen
    env.render()
#close the game
env.close()

/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


In [5]:
env.close()

ValueError: env has already been closed.

2. Preprocess Environment

In [10]:
#import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import FrameStack, GrayScaleObservation
#import Vectorization wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
#import matplot
from matplotlib import pyplot as plt

In [11]:
#1. Create the base environment
env = gym.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="human")
#2. Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
#3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
#4. wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
#5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [12]:
state = env.reset()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/zz/zyxvpxvq6csfxvn_n000046w00011q/T/ipykernel_43735/2599308756.py", line 1, in <module>
    state = env.reset()
            ^^^^^^^^^^^
  File "/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/stable_baselines3/common/vec_env/vec_frame_stack.py", line 38, in reset
    observation = self.venv.reset()  # pytype:disable=annotation-type-mismatch
                  ^^^^^^^^^^^^^^^^^
  File "/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/stable_baselines3/common/vec_env/dummy_vec_env.py", line 74, in reset
    obs = self.envs[env_idx].reset()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/gym/core.py", line 380, in reset
  Fi

In [13]:
state, reward, done, info = env.step([5])

Unexpected exception formatting exception. Falling back to standard exception


/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
Traceback (most recent call last):
  File "/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/zz/zyxvpxvq6csfxvn_n000046w00011q/T/ipykernel_43735/474621269.py", line 1, in <module>
    state, reward, done, info = env.step([5])
                                ^^^^^^^^^^^^^
  File "/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/stable_baselines3/common/vec_env/base_vec_env.py", line 163, in step
    return self.step_wait()
           ^^^^^^^^^^^^^^^^
  File "/Users/oel-bour/goinfre/anaconda3/envs/mlenv/lib/python3.11/site-packages/stable_baselines3/common/

In [14]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1, 4, idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

IndexError: tuple index out of range

<Figure size 2000x1600 with 0 Axes>

3. Train the RL Model

In [32]:
#import os for file path managment
import os
#import PPO for algos
from stable_baselines3 import PPO, A2C, A3C, DQN
#import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [33]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
    
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
    
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, f'best_model_{self.n_calls}')
            self.model.save(model_path)
        return True


In [34]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [35]:
# setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

DQN, PPO, A2C, A3C

In [ ]:
dqn = DQN('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
           n_steps=512)

In [36]:
ppo = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
           n_steps=512)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
a2c = A2C('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
           n_steps=512)

In [ ]:
a3c = A3C('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
           n_steps=512)

In [37]:
#train the agent
model.learn(total_timesteps=20000)

Logging to ./logs/PPO_1


C:\Users\elbou\anaconda3\envs\mlenv\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 24  |
|    iterations      | 1   |
|    time_elapsed    | 21  |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 27           |
|    iterations           | 2            |
|    time_elapsed         | 37           |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 5.344255e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.95        |
|    explained_variance   | 0.00178      |
|    learning_rate        | 1e-06        |
|    loss                 | 116          |
|    n_updates            | 10           |
|    policy_gradient_loss | 6.56e-05     |
|    value_loss           | 313          |
------------------------------------------
-----------------------

In [38]:
model.save("best_model_20000")

4. Test it Out

In [40]:
#load model
model = PPO.load('best_model_20000')

In [42]:
#start game
state = env.reset()
#loop through the game
while True:
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

C:\Users\elbou\anaconda3\envs\mlenv\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


KeyboardInterrupt: 